# Setup

In [1]:
# Utilities for visualizing timetables (esp. irregular ones like commuter rail)
# Requires a config.yaml with 
#   feed: (GTFS feed)
# 
# !python -m pip install yaml
import partridge as pa
import pandas as pd
# import yaml
import datetime
from collections import defaultdict
from math import floor
import sys
import numpy as np

pd.set_option('max_columns', 60)
pd.set_option('max_rows', 60)

# Interactive input
# folder = input('Enter config folder within data directory: ')
dir = 'data/' + 'mnr' + '/'

# Load config file
# config = yaml.load(open(dir + 'config.yaml'))

service_id = pa.read_service_ids_by_date(dir + 'mnrr.zip').get(datetime.date(2019, 12, 18))

view = {
    'trips.txt': {'service_id': service_id},   
}

#Load feed and tables
feed = pa.load_feed(dir + 'mnrr.zip', {'trips.txt': {'service_id': service_id}})
r = feed.routes
r['route_id'] = r['route_id'].apply(str)
r.set_index('route_id', inplace = True)

t = feed.trips
t['trip_id'] = t['trip_id'].apply(str)
t.set_index('trip_id', inplace = True)

st = feed.stop_times
st['trip_id'] = st['trip_id'].apply(str)
st.set_index(['trip_id', 'stop_sequence'], inplace=True)
st.sort_index(inplace=True)

s = feed.stops
s['stop_id'] = s['stop_id'].apply(str)
s.set_index(['stop_id'], inplace = True)

# Helper functions

In [2]:
# Convert seconds after midnight to HH:MM
def secondsToTime(seconds):
    return '{:02d}:{:02d}'.format(floor(seconds / 3600), floor(seconds % 3600 / 60))

# Grand Central has stop_id = 1
def servesGrandCentral (pattern):
    return '1' in pattern.stops

def hashStopArray(stopArray):
    result = 1
    for i in stopArray:
        result = 31 * result + hash(i)
    return result

def patternInfoForTrip(patterns, route, direction, trip):
    tripName = str(trip)
    pattern = patternNameForTrip[tripName]
    timetable = patterns.loc[route, direction, pattern]
    print(pattern, sorted(set(timetable.Trip.tolist())))

def allTripsWithSamePattern(patterns, route, direction, trip):
    tripName = str(trip)
    pattern = patternNameForTrip[tripName]
    return sorted(set(patterns.loc[route, direction, pattern].Trip.tolist()))
    
class TripPatternKey:
    
    # based on https://github.com/conveyal/gtfs-lib/blob/master/src/main/java/com/conveyal/gtfs/TripPatternKey.java
    
    def __init__(self, routeId, stops, sequences, pickups, dropoffs): 
        self.routeId = routeId
        # TODO check that lengths of these arrays are equal
        self.stops = stops
        self.sequences = sequences
        self.pickups = pickups
        self.dropoffs = dropoffs
        self.name = ''
        
    def __eq__(self, obj):
        if self.routeId != obj.routeId: return False
        if self.stops != obj.stops: return False
        if self.pickups != obj.pickups: return False
        if self.dropoffs != obj.dropoffs: return False
        return True
    
    def __hash__(self):
        result = hash(self.routeId)
        result = 31 * result + hashStopArray(self.stops)
        result = 31 * result + hashStopArray(self.pickups)
        result = 31 * result + hashStopArray(self.dropoffs)
        return result
    
    def __iter__(self):
        return iter(self.stops)
    
    def pseudoName(self, s):
        firstStop = self.stops[0]
        firstStopName = s.loc[firstStop].stop_name
        lastStop = self.stops[-1]
        lastStopName = s.loc[lastStop].stop_name
        return '{} stops from {} to {}'.format(len(self.stops), firstStopName, lastStopName)
        # return '{} stops from {} ({}) to {} ({})'.format(len(pattern.stops), firstStopName, firstStop, lastStopName, lastStop)
    
    def subset(self, start, end):
        if (start > end):
            return list(np.extract([int(stop) <= start and int(stop) >= end for stop in pattern.stops], pattern.stops))
        else:
            return list(np.extract([int(stop) >= start and int(stop) <= end for stop in pattern.stops], pattern.stops))
    
def formatTimetables(st, t, patternNameForTrip):
    # Join stop_times and trips on trip_id
    patterns = st.reset_index().merge(t, left_on = 'trip_id', right_index = True)
    patterns.rename(columns = {
        'trip_short_name': 'Trip',
        'direction_id': 'Direction',
        'route_id': 'Route'
    }, inplace = True)
    patterns['Arrival'] = patterns['arrival_time'].apply(lambda arrival: secondsToTime(float(arrival)))
    patterns['Stop'] = patterns['stop_id'].apply(lambda stopId: (stopId, s.loc[stopId].stop_name))
    patterns['Pattern'] = patterns['Trip'].apply(lambda trip: patternNameForTrip[trip])
    patterns.drop(columns = ['trip_id', 'stop_sequence', 'departure_time', 'stop_id', 'pickup_type', 'drop_off_type', 'track', 'note_id', 'service_id', 'trip_headsign', 'block_id', 'shape_id', 'wheelchair_accessible', 'peak_offpeak'], inplace = True)
    return patterns.set_index(['Route', 'Direction', 'Pattern'])

# Populate maps between pattern keys and trips
def sortTripsByPattern (st, t):
    
    tripsForPattern = defaultdict(list)
    patternForTrip = {}

    for tripId, trip in t.iterrows():
        stopTime = st.loc[tripId]
        k = TripPatternKey(
            trip['route_id'],
            stopTime.stop_id.apply(str).tolist(),
            stopTime.index.tolist(),
            stopTime.pickup_type.tolist(),
            stopTime.drop_off_type.tolist()
        )
        tripsForPattern[k].append(tripId)
        patternForTrip[tripId] = k
        
    return (tripsForPattern, patternForTrip)

# Given a map from pattern to trips following that pattern, and trip info from GTFS,
# return a map from trip to pattern name
# 
# TODO implement more sophisticated naming strategy (e.g. via stops, express/local)
# from gtfs-lib
def namePatterns(tripsForPattern, t):

    # Map from (number of stops, first stop, last stop) tuple to number of times that key has been used
    patternNamesUsed = defaultdict(int)

    # Map from trip name to its pattern name (reverse of map above, but using names)
    patternNameForTrip = {}

    for pattern, trips in tripsForPattern.items():
        patternName = pattern.pseudoName(s) +  ' like {}'.format(trips[0])
        for tripId in trips:
            trip = t.loc[tripId]
            patternNameForTrip[trip.trip_short_name] = patternName

    return patternNameForTrip

class TripsEditor:
    
    def __init__(self, s, t, st):
        self.s = s
        self.t = t
        self.st = st

    # Add a stop event to a supplied stop times file
    def addStopArrivalAfter(self, exemplarTrips, afterStopName, timeToAddedStopMinutes, addStopName, timeAddedMinutes):
        self.st.reset_index(inplace = True)        
        for exemplarTrip in exemplarTrips:

            for tripName in allTripsWithSamePattern(basePatterns, route, direction, exemplarTrip):

                tripId = self.t[self.t['trip_short_name'] == str(tripName)].index[0]
                st2 = self.st[self.st['trip_id'] == tripId]

                # TODO add warning if this stop is already served

                addStopId = self.s[self.s['stop_name'] == addStopName].index[0]
                afterStopId = int(self.s[self.s['stop_name'] == afterStopName].index[0])

                previous = st2.loc[st2['stop_id'] == str(afterStopId)]

                # Add stop_time entry
                self.st = self.st.append({
                    'trip_id': tripId,
                    'stop_id': addStopId,
                    'stop_sequence': list(previous.stop_sequence)[0] + 1,
                    'arrival_time': list(previous.arrival_time)[0] + timeToAddedStopMinutes * 60,
                    'departure_time': list(previous.departure_time)[0] + timeToAddedStopMinutes * 60
                }, ignore_index = True)

                # Shift subsequent arrivals (stop_sequence, arrival_time, and departure time)
                self.st.loc[(self.st['trip_id'] == tripId) & (self.st['stop_id'].apply(int) < int(addStopId)), 'stop_sequence'] += 1
                self.st.loc[(self.st['trip_id'] == tripId) & (self.st['stop_id'].apply(int) < int(addStopId)), 'arrival_time'] += timeAddedMinutes * 60.0
                self.st.loc[(self.st['trip_id'] == tripId) & (self.st['stop_id'].apply(int) < int(addStopId)), 'departure_time'] += timeAddedMinutes * 60.0
                
        self.st.set_index(['trip_id', 'stop_sequence'], inplace=True)
        self.st.sort_index(inplace=True)

# Process patterns

In [3]:
(tripsForPattern, patternForTrip) = sortTripsByPattern(st, t)

patternNameForTrip = namePatterns(tripsForPattern, t)

basePatterns = formatTimetables(st, t, patternNameForTrip)

# Modifications

Southbound trips have even trip names and direction_id = 1, serving stops in decreasing order
Northbound trips have odd trip names and direction_id = 0, serving stops in increasing order

## Harlem Line

In [4]:
st2 = st.reset_index()
harlemModifications = TripsEditor(s, t, st2)

route = '2'

# On the Harlem Line M-F inbound, add stops at 
# Melrose, Tremont, Botanical Garden, Williams Bridge, and Wakefield 
# on trains 314, 326, 330, 684, and 988/688.
convertToLocalHarlemInbound = [314, 326, 330, 684, 688]

direction = 1

for trip in convertToLocalHarlemInbound:
    patternInfoForTrip(basePatterns, route, direction, trip)

7 stops from Crestwood to Grand Central like 9733081 ['310', '314', '326', '518']
7 stops from Crestwood to Grand Central like 9733081 ['310', '314', '326', '518']
6 stops from Crestwood to Grand Central like 9733091 ['330']
19 stops from Southeast to Grand Central like 9733377 ['684', '688']
19 stops from Southeast to Grand Central like 9733377 ['684', '688']


/home/ansons/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: PerformanceWarning: indexing past lexsort depth may impact performance.


In [5]:
harlemModifications.addStopArrivalAfter([310, 330], 'Mt Vernon West', 2, 'Wakefield', 9)
harlemModifications.addStopArrivalAfter([684], 'Crestwood', 9, 'Wakefield', 9)
harlemModifications.addStopArrivalAfter([310, 330, 684], 'Wakefield', 2, 'Woodlawn', 0)
harlemModifications.addStopArrivalAfter([310, 330, 684], 'Woodlawn', 3, 'Williams Bridge', 0)
harlemModifications.addStopArrivalAfter([310, 330, 684], 'Williams Bridge', 2, 'Botanical Garden', 0)
harlemModifications.addStopArrivalAfter([310, 330, 684], 'Botanical Garden', 3, 'Fordham', 0)
harlemModifications.addStopArrivalAfter([310, 330, 684], 'Fordham', 2, 'Tremont', 0)
harlemModifications.addStopArrivalAfter([310, 330, 684], 'Tremont', 3, 'Melrose', 0)

/home/ansons/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: PerformanceWarning: indexing past lexsort depth may impact performance.


In [6]:
# On the Harlem Line M-F outbound, add stops at 
# Melrose, Tremont, Botanical Garden, Williams Bridge, and Wakefield 
# on trains 607, 509, 953, 365, 373, 581, 585, 589

convertToLocalHarlemOutbound = [607, 509, 953, 365, 373, 581, 585, 589]

direction = 0

for trip in convertToLocalHarlemOutbound:
    patternInfoForTrip(basePatterns, route, direction, trip)

20 stops from Grand Central to Southeast like 9733302 ['607', '609', '611']
4 stops from Grand Central to North White Plains like 9733166 ['507', '509']
11 stops from Grand Central to Wassaic like 9733568 ['953']
7 stops from Grand Central to Crestwood like 9733103 ['351', '361', '365']
11 stops from Grand Central to North White Plains like 9733117 ['373', '547', '567']
12 stops from Grand Central to North White Plains like 9733280 ['581', '585', '589', '593']
12 stops from Grand Central to North White Plains like 9733280 ['581', '585', '589', '593']
12 stops from Grand Central to North White Plains like 9733280 ['581', '585', '589', '593']


/home/ansons/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: PerformanceWarning: indexing past lexsort depth may impact performance.


In [7]:
# Executive decision: do not modify 607 (plus 609, 611) or 953

# 509 express to White Plains
# 365, 373 express to Mount Vernon West
# 581 express to Mount Vernon West via Fordham

# Note: only 509 is within 7-9, 10-4 range.

harlemModifications.addStopArrivalAfter([509, 365, 373, 581], 'Harlem-125th St.', 4, 'Melrose', 0)
harlemModifications.addStopArrivalAfter([509, 365, 373], 'Melrose', 3, 'Tremont', 0)
harlemModifications.addStopArrivalAfter([509, 365, 373], 'Tremont', 3, 'Fordham', 0)
harlemModifications.addStopArrivalAfter([581], 'Melrose', 3, 'Tremont', 2)
harlemModifications.addStopArrivalAfter([509, 365, 373, 581], 'Fordham', 2, 'Botanical Garden', 0)
harlemModifications.addStopArrivalAfter([509, 365, 373, 581], 'Botanical Garden', 3, 'Williams Bridge', 0)
harlemModifications.addStopArrivalAfter([509, 365, 373, 581], 'Williams Bridge', 3, 'Woodlawn', 0)
harlemModifications.addStopArrivalAfter([509, 365, 373, 581], 'Woodlawn', 2, 'Wakefield', 8)

/home/ansons/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: PerformanceWarning: indexing past lexsort depth may impact performance.


In [26]:
(tripsForPattern_harlem, patternForTrip_harlem) = sortTripsByPattern(harlemModifications.st, t)

patternNameForTrip_harlem = namePatterns(tripsForPattern_harlem, t)

patterns_harlem = formatTimetables(harlemModifications.st, t, patternNameForTrip_harlem)

print(s[s.stop_name == 'Fordham'].index.tolist())
print(s[s.stop_name == 'Woodlawn'].index.tolist())

noFordhamOrWoodlawnExemplarTrips = []
tripsAroundSection_harlem = defaultdict(list)

rerouteStart = 59
rerouteEnd = 56
direction = 1
route = '2'

for pattern in tripsForPattern_harlem.keys():
    if (pattern.routeId == route and servesGrandCentral(pattern)):
        if ((str(rerouteStart) not in pattern.stops) or (str(rerouteEnd) not in pattern.stops)):
            tripName = t.loc[tripsForPattern_harlem[pattern][0]].trip_short_name
            if (t.loc[tripsForPattern_harlem[pattern][0]].direction_id == direction): 
                noFordhamOrWoodlawnExemplarTrips.append(tripName)
                startRerouteAtStop = sys.maxsize
                # FIXME hack (relies on stop_id convention)
                for stop in pattern.stops:
                    stop = int(stop)
                    if (stop > rerouteStart): 
                        startRerouteAtStop = min(startRerouteAtStop, stop)
                    if (stop < rerouteEnd): 
                        endRerouteAtStop = stop
                        break
                stopsAroundSection = pattern.subset(startRerouteAtStop, endRerouteAtStop)
                tripsAroundSection_harlem[frozenset(stopsAroundSection)].append(tripName)

['56']
['59']


In [ ]:
for trips in tripsAroundSection_harlem.values():
    print(trips[0])
    for trip in trips:
        patternInfoForTrip(patterns_harlem, route, direction, trip)

    print('-----------')


In [13]:
patterns3 = patterns_harlem.reset_index().set_index('Route').loc['2'].set_index(['Direction', 'Pattern'])
patterns3.pivot(index='Stop', columns='Trip', values='Arrival').sort_index(ascending=False).fillna('')

patterns4 = patterns3[patterns3['Trip'].isin([trips[0] for trips in tripsAroundSection_harlem.values()])] \
    .pivot(index='Stop', columns='Trip', values='Arrival') \
    .sort_index(ascending=False).fillna('') \
    .reset_index()
    
patterns4.iloc[range(7,27)]

Trip,Stop,309,355,357,511,543,545,551,557,577,601,617,645,651,655
7,"(84, Mount Kisco)",,,,,,,,,,07:03,10:53,17:11,17:49,18:19
8,"(83, Chappaqua)",,,,,,,,,,06:57,10:47,17:05,,18:13
9,"(81, Pleasantville)",,,,,,,,,,06:54,10:44,,,
10,"(80, Hawthorne)",,,,,,,,,,06:50,10:40,,,
11,"(79, Mount Pleasant)",,,,,,,,,,,10:37,,,
12,"(78, Valhalla)",,,,,,,,,,06:46,10:35,,,
13,"(76, North White Plains)",08:17,,,08:55,16:09,16:39,17:28,18:18,20:13,06:42,10:31,,,
14,"(74, White Plains)",08:07,,,08:47,16:02,16:32,17:20,18:10,20:03,06:38,10:26,,17:31,
15,"(72, Hartsdale)",08:04,,,08:44,15:59,16:29,17:17,18:07,20:00,,,,,
16,"(71, Scarsdale)",08:01,,,08:41,15:56,16:26,17:14,18:04,19:57,,,,,


In [ ]:
# On the Harlem Line inbound and outbound M-F, 
# add stops at Fordham and Woodlawn on every train that doesn’t stop there currently

harlemModifications.addStopArrivalAfter([308], 'Williams Bridge', 5, 'Fordham', 0)

harlemModifications.addStopArrivalAfter([504, 642], 'Crestwood', 8, 'Woodlawn', 3)
harlemModifications.addStopArrivalAfter([504, 642], 'Woodlawn', 5, 'Fordham', 0)

harlemModifications.addStopArrivalAfter([506, 514, 662], 'Scarsdale', 12, 'Woodlawn', 3)
harlemModifications.addStopArrivalAfter([506, 514, 662], 'Woodlawn', 5, 'Fordham', 0)

# 916 is via Bedford hills, 908 is 9746058
harlemModifications.addStopArrivalAfter([516,600,602,606,616,624,626,634,652,908,916,920,926], 'White Plains', 15, 'Woodlawn', 5)
harlemModifications.addStopArrivalAfter([516,600,602,606,616,624,626,634,652,908,916,920,926], 'Woodlawn', 5, 'White Plains', 0)

harlemModifications.addStopArrivalAfter([520], 'Fleetwood', 5, 'Woodlawn', 4)
harlemModifications.addStopArrivalAfter([520], 'Woodlawn', 5, 'Fordham', 0)

harlemModifications.addStopArrivalAfter([522], 'Scarsdale', 12, 'Woodlawn', 4)
harlemModifications.addStopArrivalAfter([522], 'Woodlawn', 5, 'Fordham', 0)

harlemModifications.addStopArrivalAfter([524], 'White Plains', 15, 'Woodlawn', 3)
harlemModifications.addStopArrivalAfter([524], 'Woodlawn', 5, 'Fordham', 0)

harlemModifications.addStopArrivalAfter([528], 'Crestwood', 8, 'Woodlawn', 3)
harlemModifications.addStopArrivalAfter([528], 'Woodlawn', 5, 'Fordham', 0)

# 566 et al. after 4 PM

harlemModifications.addStopArrivalAfter([612], 'Mount Kisco', 25, 'Woodlawn', 4)
harlemModifications.addStopArrivalAfter([612], 'Woodlawn', 5, 'Fordham', 0)

harlemModifications.addStopArrivalAfter([618], 'Mt Vernon West', 4, 'Woodlawn', 4)
harlemModifications.addStopArrivalAfter([618], 'Woodlawn', 5, 'Fordham', 0)

harlemModifications.addStopArrivalAfter([620], 'North White Plains', 20, 'Woodlawn', 4)
harlemModifications.addStopArrivalAfter([620], 'Woodlawn', 5, 'Fordham', 0)

harlemModifications.addStopArrivalAfter([660], 'North White Plains', 20, 'Woodlawn', 4)
harlemModifications.addStopArrivalAfter([660], 'Woodlawn', 5, 'Fordham', 0)


# 682, 698 after 4 PM

In [8]:
(tripsForPattern_harlem, patternForTrip_harlem) = sortTripsByPattern(harlemModifications.st, t)

patternNameForTrip_harlem = namePatterns(tripsForPattern_harlem, t)

patterns_harlem = formatTimetables(harlemModifications.st, t, patternNameForTrip_harlem)


noFordhamOrWoodlawnExemplarTrips = []
tripsAroundSection_harlem = defaultdict(list)

rerouteStart = 56
rerouteEnd = 59
direction = 0
route = '2'

for pattern in tripsForPattern_harlem.keys():
    if (pattern.routeId == route and servesGrandCentral(pattern)):
        if ((str(rerouteStart) not in pattern.stops) or (str(rerouteEnd) not in pattern.stops)):
            tripName = t.loc[tripsForPattern_harlem[pattern][0]].trip_short_name
            if (t.loc[tripsForPattern_harlem[pattern][0]].direction_id == direction): 
                noFordhamOrWoodlawnExemplarTrips.append(tripName)
                startRerouteAtStop = 0
                # FIXME hack (relies on stop_id convention)
                for stop in pattern.stops:
                    stop = int(stop)
                    if (stop < rerouteStart): 
                        startRerouteAtStop = max(startRerouteAtStop, stop)
                    if (stop > rerouteEnd): 
                        endRerouteAtStop = stop
                        break
                stopsAroundSection = pattern.subset(startRerouteAtStop, endRerouteAtStop)
                tripsAroundSection_harlem[frozenset(stopsAroundSection)].append(tripName)

In [9]:
for trips in tripsAroundSection_harlem.values():
    print(trips[0])
    for trip in trips:
        patternInfoForTrip(patterns_harlem, route, direction, trip)

    print('-----------')


309
14 stops from Grand Central to North White Plains like 9733080 ['309']
-----------
355
9 stops from Grand Central to Crestwood like 9733106 ['355']
-----------
357
6 stops from Grand Central to Crestwood like 9733107 ['357']
-----------
511
16 stops from Grand Central to North White Plains like 9733172 ['511', '513']
-----------
543
11 stops from Grand Central to North White Plains like 9733222 ['543']
-----------
545
7 stops from Grand Central to North White Plains like 9733224 ['545']
15 stops from Grand Central to Southeast like 9733319 ['627', '631', '635']
19 stops from Grand Central to Southeast like 9733376 ['683', '687']
-----------
551
6 stops from Grand Central to North White Plains like 9733233 ['551', '559', '563']
14 stops from Grand Central to Southeast like 9733310 ['615']
11 stops from Grand Central to Mount Kisco like 9733332 ['639']
11 stops from Grand Central to Mount Kisco like 9733346 ['653']
18 stops from Grand Central to Southeast like 9733374 ['681']
-------

/home/ansons/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: PerformanceWarning: indexing past lexsort depth may impact performance.


In [14]:
patterns3 = patterns_harlem.reset_index().set_index('Route').loc['2'].set_index(['Direction', 'Pattern'])
patterns3.pivot(index='Stop', columns='Trip', values='Arrival').sort_index(ascending=False).fillna('')

patterns4 = patterns3[patterns3['Trip'].isin([trips[0] for trips in tripsAroundSection_harlem.values()])] \
    .pivot(index='Stop', columns='Trip', values='Arrival') \
    .sort_index(ascending=True).fillna('') \
    .reset_index()
    
patterns4 #.iloc[range(7,27)]

Trip,Stop,309,355,357,511,543,545,551,557,577,601,617,645,651,655
0,"(1, Grand Central)",07:18,17:09,17:38,07:56,15:19,15:55,16:44,17:35,19:28,06:02,09:51,16:18,16:57,17:27
1,"(4, Harlem-125th St.)",07:28,17:19,,08:06,15:29,16:05,16:54,,,06:12,10:01,16:28,,
2,"(54, Melrose)",,,,08:11,,,,,,,,,,
3,"(55, Tremont)",,,,08:13,,,,,,,,,,
4,"(56, Fordham)",07:36,,,08:16,,,,,,06:20,,,,
5,"(57, Botanical Garden)",07:39,,,08:19,,,,,,,,,,
6,"(59, Woodlawn)",,17:30,,,,,,,,,,,,
7,"(61, Wakefield)",07:45,17:33,,08:25,,,,,,,,,,
8,"(62, Mt Vernon West)",07:48,17:36,18:01,08:28,15:43,,,,,,,,,
9,"(64, Fleetwood)",07:50,17:38,18:03,08:30,15:45,,,,,,,,,


In [ ]:
harlemModifications.addStopArrivalAfter([309, 511], 'Botanical Garden', 5, 'Woodlawn', 1)

# 355, 357 after 4 PM

harlemModifications.addStopArrivalAfter([543], 'Harlem-125th St.', 8, 'Fordham', 4)
harlemModifications.addStopArrivalAfter([543], 'Fordham', 6, 'Woodlawn', 0)

harlemModifications.addStopArrivalAfter([545, 627, 683], 'Harlem-125th St.', 8, 'Fordham', 4)
harlemModifications.addStopArrivalAfter([545, 627, 683], 'Fordham', 6, 'Woodlawn', 0)

harlemModifications.addStopArrivalAfter([551, 615, 639, 653, 681], 'Harlem-125th St.', 8, 'Fordham', 4)
harlemModifications.addStopArrivalAfter([551, 615, 639, 653, 681], 'Fordham', 6, 'Woodlawn', 0)

# 557, 577 after 4 PM

harlemModifications.addStopArrivalAfter([601], 'Fordham', 6, 'Woodlawn', 2)

harlemModifications.addStopArrivalAfter([617,621,637,657,663,665,667,943,949,953], 'Harlem-125th St.', 8, 'Fordham', 4)
harlemModifications.addStopArrivalAfter([617,621,637,657,663,665,667,943,949,953], 'Fordham', 6, 'Woodlawn', 0)

# 645, 651 , 655 after 4 PM

In [82]:
SCRATCH


stopNames = {}

for (index, stop) in s.reset_index().iterrows():
    stopNames[int(stop.stop_id)] = stop.stop_name

#patterns4['Stop'] = patterns4['Stop'].map(stopNames)












        
        # TODO replace this inner loop w/ dataframe joins
        for sequence in pattern.sequences:
            stopTime = st.loc[tripId, sequence]
            stopId = stopTime.stop_id
            stopName = s.loc[stopId].stop_name
            patterns = patterns.append({
                'Route': pattern.routeId, 
                'Direction': trip.direction_id,
                'Pattern': patternName,
                'Trip': trip.trip_short_name,
                # 'Sequence': pattern.sequences[i],
                # We should use sequence for ordering, but instead rely on MNR conveniently 
                # setting stop_ids sequentially along lines
                'Stop': (int(stopId), stopName),
                'Arrival': secondsToTime(stopTime.arrival_time)}
            , ignore_index=True)
            
basePatterns = patterns.set_index(['Route', 'Direction', 'Pattern'])

IndentationError: unexpected indent (<ipython-input-82-082a88a67dd3>, line 21)